Importando as bibliotecas

In [1]:
import numpy as np
import pandas as pd

Carregando base de dados

In [2]:
# Treinamento com a variável alvo
transact_train_database = pd.read_csv('data/transact_train.txt', sep='|')

# Teste sem a variável alvo
transact_test_database = pd.read_csv('data/transact_class.txt', sep = '|')

# Variável alvo do teste
transact_original_database = pd.read_csv('data/realclass_t1.txt', sep = '|')

transact_train_database.head()

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder,order
0,1,6,5,0.000,1,59.99,59.99,59.99,1,59.99,...,?,1,600,70,21,1,43,1,49,y
1,1,6,5,11.940,1,59.99,59.99,59.99,1,59.99,...,completely orderable,1,600,70,21,1,43,1,49,y
2,1,6,5,39.887,1,59.99,59.99,59.99,1,59.99,...,completely orderable,1,600,70,21,1,43,1,49,y
3,2,6,5,0.000,0,?,?,?,0,?,...,completely orderable,?,?,?,?,?,?,?,?,y
4,2,6,5,15.633,0,?,?,?,0,?,...,completely orderable,?,?,?,?,?,?,?,?,y


Alterando a granularidade

In [3]:
transact_train_database = transact_train_database.drop_duplicates(subset=['sessionNo'], keep='last')
transact_train_database_y = transact_train_database['order']
transact_train_database_y = transact_train_database_y.replace({'y': 1, 'n': 0})
transact_train_database_x = transact_train_database.drop(['order'], axis=1)

transact_test_database_x = transact_test_database.drop_duplicates(subset=['sessionNo'], keep='last')

In [4]:
transact_train_database_y.head()

2     1
7     1
12    1
14    0
22    1
Name: order, dtype: int64

In [5]:
transact_test_database_x.head()

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,onlineStatus,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder
8,1,18,7,624.606,11,16.99,39.99,207.91,5,16.99,...,y,completely orderable,25039,1300,489,188,5,49,1,65
19,2,18,7,2804.705,16,34.99,34.99,174.95,2,34.99,...,y,completely orderable,25040,1200,543,43,5,29,2,184
42,3,18,7,7401.384,119,7.99,59.95,3263.57,12,12.49,...,y,completely orderable,25041,600,552,17,4,37,2,107
48,4,18,7,2853.550,152,3.99,239.99,5642.5,4,9.99,...,?,?,25042,8500,535,226,19,49,2,17
50,5,18,7,48.145,2,29.99,29.99,59.98,1,29.99,...,y,completely orderable,25043,600,543,39,2,53,2,234


Tratando os valores ausentes

Lista de variáveis numéricas

In [6]:
list_numeric_variables = [
  'cMinPrice',
  'cMaxPrice',
  'cSumPrice',
  'bMinPrice',
  'bMaxPrice',
  'bSumPrice',
  'bStep',
  'maxVal',
  'customerScore',
  'accountLifetime',
  'payments',
  'age',
  'address',
  'lastOrder',
]

Lista de variáveis string

In [7]:
list_string_variables = [
  'availability',
  'onlineStatus',
]

In [8]:
list_customer_variables = [
  'maxVal',
  'customerScore',
  'accountLifetime',
  'payments',
  'age',
  'address',
  'lastOrder',
]

In [9]:
def replaceValueMissing(df, columns, value):
  removed = df[columns].replace(value, np.nan)
  for column in columns:
    df[column] = removed[column]

  return df

In [10]:
def convertFloat(df, columns):
  for column in columns:
    df[column] = df[column].astype(float)

  return df

In [11]:
def replaceMissingByMean(df, columns):
  for column in columns:
    mean = df[column].mean(axis=0)
    df[column].fillna(mean, inplace=True)

  return df

In [12]:
def replaceMissingByMode(df, columns):
  for column in columns:
    mode = df[column].mode(axis=0)
    df[column].fillna(mode[0], inplace=True)

  return df

In [13]:
def replaceMissingByMedian(df, columns):
  for column in columns:
    median = df[column].median(axis=0)
    df[column].fillna(median, inplace=True)

  return df

In [14]:
def replaceMissingByFixedValue(df, columns, value):
  for column in columns:
    df[column].fillna(value, inplace=True)

  return df

Base na granularidade do projeto

In [15]:
# Substituindo a string ? por np.nan em todas p/ todas as variáveis da base na granularidade do projeto

transact_train_database_x = replaceValueMissing(transact_train_database_x.copy(), list_numeric_variables, '?')
transact_train_database_x = replaceValueMissing(transact_train_database_x.copy(), list_string_variables, '?')

transact_test_database_x = replaceValueMissing(transact_test_database_x.copy(), list_numeric_variables, '?')
transact_test_database_x = replaceValueMissing(transact_test_database_x.copy(), list_string_variables, '?')

transact_train_database_x = convertFloat(transact_train_database_x, list_numeric_variables)
transact_test_database_x = convertFloat(transact_test_database_x, list_numeric_variables)

transact_train_database_x = replaceMissingByFixedValue(transact_train_database_x, list_string_variables, 'ausente')
transact_test_database_x = replaceMissingByFixedValue(transact_test_database_x, list_string_variables, 'ausente')

# Altera também na base original

transact_train_database = replaceValueMissing(transact_train_database, list_string_variables, '?')
transact_train_database = replaceValueMissing(transact_train_database, list_numeric_variables, '?')
transact_train_database = convertFloat(transact_train_database, list_numeric_variables)

transact_test_database = replaceValueMissing(transact_test_database, list_string_variables, '?')
transact_test_database = replaceValueMissing(transact_test_database, list_numeric_variables, '?')
transact_test_database = convertFloat(transact_test_database, list_numeric_variables)

In [16]:
transact_train_database_x.head()

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,onlineStatus,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder
2,1,6,5,39.887,1,59.99,59.99,59.99,1,59.99,...,y,completely orderable,1,600.0,70.0,21.0,1.0,43.0,1.0,49.0
7,2,6,5,94.469,0,NaN,NaN,NaN,0,NaN,...,y,completely orderable,?,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,3,6,5,341.613,11,9.99,29.99,109.95,2,9.99,...,y,completely orderable,3,1800.0,475.0,302.0,12.0,45.0,1.0,11.0
14,4,6,5,42.812,4,4.99,4.99,19.96,1,4.99,...,y,completely not orderable,?,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,5,6,5,2816.046,45,12.99,179.95,1093.72,4,19.99,...,y,completely orderable,4,800.0,503.0,18.0,1.0,46.0,1.0,40.0


In [17]:
transact_train_database_x_by_mean = replaceMissingByMean(transact_train_database_x.copy(), list_numeric_variables)
transact_test_database_x_by_mean = replaceMissingByMean(transact_test_database_x.copy(), list_numeric_variables)

transact_train_database_x_by_mean = replaceValueMissing(transact_train_database_x_by_mean, list_customer_variables, '?')
transact_test_database_x_by_mean = replaceValueMissing(transact_test_database_x_by_mean, list_customer_variables, '?')

transact_test_database_x_by_mean = replaceMissingByMean(transact_test_database_x.copy(), list_customer_variables)
transact_train_database_x_by_mean = replaceMissingByMean(transact_train_database_x.copy(), list_customer_variables)

In [18]:
transact_test_database_x_by_mean.to_csv('data/transact_test_database_x_by_mean.csv', index=False)
transact_train_database_x_by_mean.to_csv('data/transact_train_database_x_by_mean.csv', index=False)
transact_train_database_y.copy().to_csv('data/transact_train_database_y_by_mean.csv', index=False, header=False)

In [19]:
transact_train_database_x_by_mean.head()

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,onlineStatus,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder
2,1,6,5,39.887,1,59.99,59.99,59.99,1,59.99,...,y,completely orderable,1,600.000000,70.000000,21.000000,1.000000,43.000000,1.000000,49.000000
7,2,6,5,94.469,0,NaN,NaN,NaN,0,NaN,...,y,completely orderable,?,2266.582859,483.731568,128.169304,12.893535,44.921048,1.706919,79.007675
12,3,6,5,341.613,11,9.99,29.99,109.95,2,9.99,...,y,completely orderable,3,1800.000000,475.000000,302.000000,12.000000,45.000000,1.000000,11.000000
14,4,6,5,42.812,4,4.99,4.99,19.96,1,4.99,...,y,completely not orderable,?,2266.582859,483.731568,128.169304,12.893535,44.921048,1.706919,79.007675
22,5,6,5,2816.046,45,12.99,179.95,1093.72,4,19.99,...,y,completely orderable,4,800.000000,503.000000,18.000000,1.000000,46.000000,1.000000,40.000000


In [20]:
# Atribuindo valor zero para as variáveis comportamentais que não possuem valor de customerNo

rows_without_customerNo = transact_train_database_x_by_mean['customerNo'] == '?'
transact_train_database_x_by_mean.loc[rows_without_customerNo, 'maxVal'] = 0
transact_train_database_x_by_mean.loc[rows_without_customerNo, 'customerScore'] = 0
transact_train_database_x_by_mean.loc[rows_without_customerNo, 'accountLifetime'] = 0
transact_train_database_x_by_mean.loc[rows_without_customerNo, 'payments'] = 0
transact_train_database_x_by_mean.loc[rows_without_customerNo, 'age'] = 0
transact_train_database_x_by_mean.loc[rows_without_customerNo, 'address'] = 0
transact_train_database_x_by_mean.loc[rows_without_customerNo, 'lastOrder'] = 0

# Teste
rows_test_without_customerNo = transact_test_database_x_by_mean['customerNo'] == '?'
transact_test_database_x_by_mean.loc[rows_test_without_customerNo, 'maxVal'] = 0
transact_test_database_x_by_mean.loc[rows_test_without_customerNo, 'customerScore'] = 0
transact_test_database_x_by_mean.loc[rows_test_without_customerNo, 'accountLifetime'] = 0
transact_test_database_x_by_mean.loc[rows_test_without_customerNo, 'payments'] = 0
transact_test_database_x_by_mean.loc[rows_test_without_customerNo, 'age'] = 0
transact_test_database_x_by_mean.loc[rows_test_without_customerNo, 'address'] = 0
transact_test_database_x_by_mean.loc[rows_test_without_customerNo, 'lastOrder'] = 0

In [21]:
transact_train_database_x.describe(include='all')

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,onlineStatus,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,49682.000000,49682.000000,49682.000000,50000.000000,49325.000000,...,50000,50000,50000,27046.000000,27046.000000,27046.000000,27361.000000,27333.000000,27361.000000,27361.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3,8,25038,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,y,completely orderable,?,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38684,36277,22639,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,25000.500000,14.367060,5.923140,1465.728502,20.233700,72.118160,157.646205,1096.642791,3.207580,86.444819,...,NaN,NaN,NaN,2266.582859,483.731568,128.169304,12.893535,44.921048,1.706919,79.007675
std,14433.901067,4.638547,0.795063,2314.980566,28.430273,176.181174,295.331281,3083.223860,3.549983,203.739233,...,NaN,NaN,NaN,2755.134822,131.400171,108.671163,26.743458,12.083048,0.457506,112.447109
min,1.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,17.000000,1.000000,3.000000
25%,12500.750000,11.000000,5.000000,217.926750,4.000000,7.000000,24.990000,119.960000,1.000000,9.990000,...,NaN,NaN,NaN,600.000000,480.000000,36.000000,2.000000,36.000000,1.000000,14.000000
50%,25000.500000,15.000000,6.000000,686.784500,10.000000,14.990000,49.990000,358.935000,2.000000,19.990000,...,NaN,NaN,NaN,1300.000000,520.000000,98.000000,8.000000,45.000000,2.000000,33.000000
75%,37500.250000,18.000000,7.000000,1726.262750,24.000000,39.990000,129.990000,938.637500,4.000000,49.990000,...,NaN,NaN,NaN,3200.000000,553.000000,215.000000,14.000000,53.000000,2.000000,85.000000


In [22]:
transact_test_database_x.describe(include='all')

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,onlineStatus,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder
count,5111.000000,5111.000000,5111.00000,5111.000000,5111.000000,5079.000000,5079.000000,5079.000000,5111.000000,5042.000000,...,5111,5111,5111,2541.000000,2541.000000,2541.000000,2573.000000,2569.000000,2573.000000,2573.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3,7,2441,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,y,completely orderable,?,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3893,3694,2538,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,2556.000000,17.996087,6.23107,1527.974712,23.304637,72.880350,171.776058,1267.255050,3.395813,89.261148,...,NaN,NaN,NaN,1929.004329,476.621409,120.714286,9.496308,43.525107,1.712398,89.313253
std,1475.562943,6.365912,2.00577,2184.293690,31.657279,167.712913,304.336070,3753.375477,3.641097,195.863501,...,NaN,NaN,NaN,2079.223468,142.763951,104.856465,13.502393,11.876681,0.456155,122.612305
min,1.000000,0.000000,1.00000,0.000000,0.000000,0.000000,1.000000,3.000000,0.000000,0.000000,...,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,17.000000,1.000000,4.000000
25%,1278.500000,19.000000,7.00000,206.114500,4.000000,6.990000,29.990000,134.905000,1.000000,9.225000,...,NaN,NaN,NaN,600.000000,478.000000,31.000000,2.000000,34.000000,1.000000,16.000000
50%,2556.000000,20.000000,7.00000,793.389000,11.000000,12.990000,49.990000,399.970000,2.000000,17.990000,...,NaN,NaN,NaN,1000.000000,520.000000,87.000000,7.000000,44.000000,2.000000,37.000000
75%,3833.500000,21.000000,7.00000,1966.657000,30.000000,44.990000,159.990000,1089.295000,4.000000,49.990000,...,NaN,NaN,NaN,3000.000000,554.000000,207.000000,12.000000,52.000000,2.000000,105.000000


In [23]:
transact_train_database.head()

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder,order
2,1,6,5,39.887,1,59.99,59.99,59.99,1,59.99,...,completely orderable,1,600.0,70.0,21.0,1.0,43.0,1.0,49.0,y
7,2,6,5,94.469,0,NaN,NaN,NaN,0,NaN,...,completely orderable,?,NaN,NaN,NaN,NaN,NaN,NaN,NaN,y
12,3,6,5,341.613,11,9.99,29.99,109.95,2,9.99,...,completely orderable,3,1800.0,475.0,302.0,12.0,45.0,1.0,11.0,y
14,4,6,5,42.812,4,4.99,4.99,19.96,1,4.99,...,completely not orderable,?,NaN,NaN,NaN,NaN,NaN,NaN,NaN,n
22,5,6,5,2816.046,45,12.99,179.95,1093.72,4,19.99,...,completely orderable,4,800.0,503.0,18.0,1.0,46.0,1.0,40.0,y


In [24]:
transact_test_database.head()

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,onlineStatus,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder
0,1,18,7,136.833,3,39.99,39.99,79.98,1,39.99,...,y,completely orderable,25039,1300.0,489.0,188.0,5.0,49.0,1.0,65.0
1,1,18,7,189.984,3,39.99,39.99,79.98,1,39.99,...,y,completely orderable,25039,1300.0,489.0,188.0,5.0,49.0,1.0,65.0
2,1,18,7,342.894,6,16.99,39.99,113.96,2,16.99,...,NaN,NaN,25039,1300.0,489.0,188.0,5.0,49.0,1.0,65.0
3,1,18,7,411.051,8,16.99,39.99,149.94,3,16.99,...,NaN,NaN,25039,1300.0,489.0,188.0,5.0,49.0,1.0,65.0
4,1,18,7,460.049,10,16.99,39.99,189.92,4,16.99,...,NaN,NaN,25039,1300.0,489.0,188.0,5.0,49.0,1.0,65.0


Criando novas variáveis

In [25]:
transact_train_database_x_by_mean = transact_train_database_x_by_mean.set_index('sessionNo')
transact_test_database_x_by_mean = transact_test_database_x_by_mean.set_index('sessionNo')

In [26]:
# Criando variável bSumPrice_cSumPrice
transact_test_database_x_by_mean['bSumPrice_cSumPrice'] = transact_test_database_x_by_mean['bSumPrice'] / transact_test_database_x_by_mean['cSumPrice']
transact_train_database_x_by_mean['bSumPrice_cSumPrice'] = transact_train_database_x_by_mean['bSumPrice'] / transact_train_database_x_by_mean['cSumPrice']

# --------- Definindo lastOrder_accountLifetime ---------


In [27]:
# Criando variável lastOrder_accountLifetime
transact_test_database_x_by_mean['lastOrder_accountLifetime'] = transact_test_database_x_by_mean['lastOrder'] / transact_test_database_x_by_mean['accountLifetime']
transact_train_database_x_by_mean['lastOrder_accountLifetime'] = transact_train_database_x_by_mean['lastOrder'] / transact_train_database_x_by_mean['accountLifetime']


In [28]:
# Criando variável bCount_cCount
transact_test_database_x_by_mean['bCount_cCount'] = transact_test_database_x_by_mean['bCount'] / transact_test_database_x_by_mean['cCount']
transact_train_database_x_by_mean['bCount_cCount'] = transact_train_database_x_by_mean['bCount'] / transact_train_database_x_by_mean['cCount']


In [29]:
transact_test_database_x_by_mean.head()

,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,bMaxPrice,...,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder,bSumPrice_cSumPrice,lastOrder_accountLifetime,bCount_cCount
sessionNo,,,,,,,,,,,,,,,,,,,,,
1,18,7,624.606,11,16.99,39.99,207.91,5,16.99,39.99,...,1300.0,489.0,188.0,5.0,49.0,1.0,65.0,0.543264,0.345745,0.454545
2,18,7,2804.705,16,34.99,34.99,174.95,2,34.99,34.99,...,1200.0,543.0,43.0,5.0,29.0,2.0,184.0,0.400000,4.279070,0.125000
3,18,7,7401.384,119,7.99,59.95,3263.57,12,12.49,39.95,...,600.0,552.0,17.0,4.0,37.0,2.0,107.0,0.106190,6.294118,0.100840
4,18,7,2853.550,152,3.99,239.99,5642.50,4,9.99,14.99,...,8500.0,535.0,226.0,19.0,49.0,2.0,17.0,0.007968,0.075221,0.026316
5,18,7,48.145,2,29.99,29.99,59.98,1,29.99,29.99,...,600.0,543.0,39.0,2.0,53.0,2.0,234.0,0.500000,6.000000,0.500000


In [30]:
transact_train_database_x_by_mean.head()

,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,bMaxPrice,...,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder,bSumPrice_cSumPrice,lastOrder_accountLifetime,bCount_cCount
sessionNo,,,,,,,,,,,,,,,,,,,,,
1,6,5,39.887,1,59.99,59.99,59.99,1,59.99,59.99,...,600.0,70.0,21.0,1.0,43.0,1.0,49.0,1.000000,2.333333,1.000000
2,6,5,94.469,0,NaN,NaN,NaN,0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
3,6,5,341.613,11,9.99,29.99,109.95,2,9.99,29.99,...,1800.0,475.0,302.0,12.0,45.0,1.0,11.0,0.363620,0.036424,0.181818
4,6,5,42.812,4,4.99,4.99,19.96,1,4.99,4.99,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250000,NaN,0.250000
5,6,5,2816.046,45,12.99,179.95,1093.72,4,19.99,27.85,...,800.0,503.0,18.0,1.0,46.0,1.0,40.0,0.094668,2.222222,0.088889


# Transformação dos Dados

In [31]:
#  Remove variáveis contendo ids
transact_train_database_x_by_mean = transact_train_database_x_by_mean.drop('customerNo', axis=1)
transact_test_database_x_by_mean = transact_test_database_x_by_mean.drop('customerNo', axis=1)

In [32]:
# Criação de variáveis dummies
transact_train_database_x_by_mean = pd.get_dummies(transact_train_database_x_by_mean, prefix='_')
transact_test_database_x_by_mean = pd.get_dummies(transact_test_database_x_by_mean, prefix='_')

In [33]:
assert(transact_train_database_x_by_mean.shape[1] == transact_test_database_x_by_mean.shape[1]), 'Número de variáveis diferente'

AssertionError: Número de variáveis diferente

In [ ]:
variables_transact_train_database_x = list(transact_train_database_x_by_mean.columns)
variables_transact_test_database_x = list(transact_test_database_x_by_mean.columns)

in_train_not_test = [i for i in variables_transact_train_database_x if i not in variables_transact_test_database_x]
in_test_not_train = [i for i in variables_transact_test_database_x if i not in variables_transact_train_database_x]

if len(in_train_not_test) > 0:
    print('Variáveis que tão no treinamneto e não no teste')
    print(in_train_not_test)
else:
    print('Todas as variáveis do treinamento estão no teste')

if len(in_test_not_train) > 0:
    print('Variáveis que tão no teste e não no treinamento')
    print(in_test_not_train)
else:
    print('Todas as variáveis do teste estão no treinamento')

In [ ]:
transact_train_database_x_by_mean['__mainly not determinable'] = transact_train_database_x_by_mean['__mainly not determinable'] + transact_train_database_x_by_mean['__mainly not determinable']
transact_train_database_x_by_mean = transact_train_database_x_by_mean.drop('__mainly not determinable', axis=1)

In [ ]:
assert(transact_train_database_x_by_mean.shape[1] == transact_test_database_x_by_mean.shape[1]), 'Número de variáveis diferente'

In [ ]:
# Checando se todas as colunas estão como numericas, ou seja, não existem variáveis String (type object)
columns_obj = transact_train_database_x_by_mean.dtypes[transact_train_database_x_by_mean.dtypes == np.object]

assert(len(columns_obj) == 0), 'Existem variáveis do tipo object'

columns_obj_test = transact_test_database_x_by_mean.dtypes[transact_test_database_x_by_mean.dtypes == np.object]

assert(len(columns_obj_test) == 0), 'Existem variáveis do tipo object'

print('Passou no teste: as bases não possuem variáveis String!')

# Salvando as bases de dados

In [ ]:
transact_train_database_x_by_mean.to_csv('transact_train_database_x_by_mean.csv', index=False)
transact_test_database_x_by_mean.to_csv('transact_test_database_x_by_mean.csv', index=False)
transact_train_database_x_by_mean.to_csv('transact_train_database_x_by_mean.csv', index=False)
transact_train_database_y.to_csv('transact_train_database_y_by_mean.csv', index=False, header=True)